# Python Selenium DFB (Don't Follow Back)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
%matplotlib inline


# Setting the Firefox Web Driver ----------------------------------------------

driver = webdriver.Firefox()
driver.maximize_window()


# Setting the Chrome Web Driver -----------------------------------------------
"""
options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(options=options)
"""


# Login -----------------------------------------------------------------------

username = ''
password = ''

driver.get('https://www.instagram.com/accounts/login/')

time.sleep(2)

driver.find_element_by_name('username').send_keys(username)
driver.find_element_by_name('password').send_keys(password)
driver.find_element_by_css_selector('button[type="submit"]').click()


# Changing to profile page ----------------------------------------------------

time.sleep(1)

driver.get('https://www.instagram.com/%s' % username)

time.sleep(1)


# Checking the followers ------------------------------------------------------

followers_link = driver.find_element_by_css_selector('a[href="/%s/followers/"]' % username)
followers_qty = int(followers_link.find_element_by_css_selector('span').text)
followers_link.click()

time.sleep(2)

followers_container = driver.find_element_by_css_selector('div[class="isgrP"]')
followers_list = driver.find_element_by_css_selector('div[role="dialog"] ul').get_attribute('innerHTML')

time.sleep(2)

while followers_list.count('</li>') < followers_qty:
    driver.execute_script('arguments[0].scrollTop += 250', followers_container)
    time.sleep(0.5)
    followers_list = driver.find_element_by_css_selector('div[role="dialog"] ul').get_attribute('innerHTML')

webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()


# Checking who you follow -----------------------------------------------------

following_link = driver.find_element_by_css_selector('a[href="/%s/following/"]' % username)
following_qty = int(following_link.find_element_by_css_selector('span').text)
following_link.click()

time.sleep(2)

following_container = driver.find_element_by_css_selector('div[class="isgrP"]')
following_list = driver.find_element_by_css_selector('div[role="dialog"] ul').get_attribute('innerHTML')

time.sleep(2)

while following_list.count('</li>') < following_qty:
    driver.execute_script('arguments[0].scrollTop += 250', following_container)
    time.sleep(0.5)
    following_list = driver.find_element_by_css_selector('div[role="dialog"] ul').get_attribute('innerHTML')

webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()


# Logout ----------------------------------------------------------------------

time.sleep(0.5)

driver.get('https://www.instagram.com/accounts/logout/')

time.sleep(1)


# Close the browser -----------------------------------------------------------

driver.quit()


# Beautiful Soup - Followers --------------------------------------------------

soup = BeautifulSoup(followers_list, 'html.parser')

formated_followers_list = []

for li in soup.find_all("li"):
    formated_followers_list.append(li.a['href'][1:-1])


# Beautiful Soup - Following --------------------------------------------------

soup = BeautifulSoup(following_list, 'html.parser')

formated_following_list = []

for li in soup.find_all("li"):
    formated_following_list.append(li.a["href"][1:-1])


# Beautiful Soup - Don't Follow Back ------------------------------------------

dont_follow_back_list = list(set(formated_following_list) - set(formated_followers_list))


# Formating results ------------------------------------------------------------

qty_formated_followers_list = len(formated_followers_list)
qty_formated_following_list = len(formated_following_list)
qty_dont_follow_back_list = len(dont_follow_back_list)

leg_followers = 'Followers (%s)' % qty_formated_followers_list
leg_following = 'Following (%s)' % qty_formated_following_list
leg_dont_follow_back = 'DFB (%s)' % qty_dont_follow_back_list

max = qty_formated_followers_list if qty_formated_followers_list > qty_formated_following_list else qty_formated_following_list

i = 0
while i < max:
    if i >= len(formated_followers_list):
        formated_followers_list.append('')
    if i >= len(formated_following_list):
        formated_following_list.append('')
    if i >= len(dont_follow_back_list):
        dont_follow_back_list.append('')
    i += 1

mat = np.array([formated_followers_list, formated_followers_list, dont_follow_back_list]).T

df = pd.DataFrame(mat, columns=[leg_followers, leg_following, leg_dont_follow_back])
df.index = pd.Index(range(1, max+1))


# Chart ------------------------------------------------------------------------

objects = (leg_followers, leg_following, leg_dont_follow_back)
y_pos = np.arange(len(objects))
performance = [qty_formated_followers_list,qty_formated_following_list,qty_dont_follow_back_list]

plt.bar(y_pos, performance, align='center', alpha=0.5)
plt.xticks(y_pos, objects)
plt.ylabel('Quantity')
plt.title('@%s' % username)
 
plt.show()


# Data -------------------------------------------------------------------------

print(df)